In [1]:
import pandas as pd

df = pd.read_csv('/Users/JooAnLee/mid_project/src/datasets/수도권_아파트_청약_뉴스_크롤링.csv')

df.head()

,title,content,url,apartment
0,대기업도 미분양 폭탄인데…'10만 청약' 몰린 대방 디에트르의 비결,이밖에 11월 경기 군포시 대야미지구에 분양한 ‘군포대야미 대방 디에트르 시그니처’...,https://realty.chosun.com/site/data/html_dir/2...,의왕 월암지구 대방 디에트르 레이크파크
1,“7인 가구에 무주택 15년 이상”…새해 첫 만점통장 ‘이곳’서 나왔다,13일 청약홈에 따르면 지난 8일 당첨자를 발표한 ‘의왕 월암지구 대방 디에트르 레...,https://www.mk.co.kr/article/11216596,의왕 월암지구 대방 디에트르 레이크파크
2,[이번주 분양일정]더팰리스트데시앙外 2천177가구 분양,경기 의왕시 청계동 ‘의왕청계2A1블록신혼희망타운(공공분양)’ 등에서 청약을 진행한...,http://www.kjdaily.com/article.php?aid=1736694...,의왕 월암지구 대방 디에트르 레이크파크
3,올해 첫 청약 통장 만점 나온 단지는…서울 아니고 이곳,11일 청약홈에 따르면 지난 8일 당첨자를 발표한 '의왕 월암지구 대방 디에트르 레...,https://www.sedaily.com/NewsView/2GNNIIBZDD,의왕 월암지구 대방 디에트르 레이크파크
4,"""지금 아니면 내집마련 못한다?""...새해 첫 84점 만점통장 나왔다",11일 청약홈에 따르면 지난 8일 당첨자를 발표한 경기 의왕시 '의왕 월암지구 대방...,http://www.fnnews.com/news/202501101414229320,의왕 월암지구 대방 디에트르 레이크파크


In [2]:
content_0 = df[df['apartment'] == '의왕 월암지구 대방 디에트르 레이크파크'].iloc[0]['content']
content_0

'이밖에 11월 경기 군포시 대야미지구에 분양한 ‘군포대야미 대방 디에트르 시그니처’가 경쟁률 7.74대 1, 12월 의왕시 월암지구에 공급한 ‘의왕 월암지구 대방 디에트르 레이크파크’가 4.2대 1 경쟁률을 기록하는 등 성적을 보였다. 월암지구에서도 만점 청약통장이 등장했다. ■ 올해 7000가구 분양 예고... '

# 라이브러리 설치

In [3]:
!pip install konlpy
!apt-get install -y default-jdk

zsh:1: command not found: apt-get


In [4]:
!pip install soynlp

import re
from konlpy.tag import Okt
from soynlp.word import WordExtractor

In [5]:
pip install soynlp==0.0.493

Note: you may need to restart the kernel to use updated packages.


In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] =False

# 텍스트 전처리

In [8]:
# SoyNLP로 고유명사 추출
def extract_proper_nouns(content_0):
    extractor = WordExtractor()
    extractor.train([content_0])
    word_scores = extractor.extract()
    return [word for word, score in word_scores.items() if score.cohesion_forward > 0.3]

In [13]:
from konlpy.tag import Okt
import re

okt = Okt()
stopwords = ['곳', '외', '것', '도', '이', '명']

def preprocess_korean(content_0):
    # 1단계: 기본 전처리
    text_clean = re.sub(r'[^가-힣0-9\s]', '', content_0)
    
    # 2단계: SoyNLP 고유명사 추출
    proper_nouns = extract_proper_nouns(text_clean)
    
    # 3단계: 숫자+한글 결합 단어 추출 (예: 84㎡, 7호선)
    num_hangul = re.findall(r'\d+[가-힣㎡]+', text_clean)
    
    # 4단계: Okt 명사 추출 (고유명사 반영)
    okt_nouns = okt.nouns(text_clean)
    
    # 5단계: 통합 및 필터링
    combined = list(set(proper_nouns + num_hangul + okt_nouns))
    return [word for word in combined if word not in stopwords and len(word) > 1]

result = preprocess_korean(content_0)
print(result)

training was done. used memory 0.036 Gb0.035 Gb
all cohesion probabilities was computed. # words = 0
all branching entropies was computed # words = 8
all accessor variety was computed # words = 8
['청약', '레이크', '올해', '군포시', '통장', '공급', '11월', '경쟁률', '분양', '대야', '성적', '처가', '디에트르', '의왕시', '대방', '예고', '만점', '774대', '파크', '7000가구', '의왕', '경기', '가구', '42대', '등장', '군포', '지구', '기록', '12월']


# TF-IDF

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

# 병합된 텍스트를 다시 하나의 문자열로 변환
processed_text = ' '.join(result)

# TF-IDF 벡터화기기 생성
vectorizer = TfidfVectorizer(
    max_features=1000,      # 상위 1000개 단어까지만 저장
    use_idf=True            # ✅ IDF 활성화
)

# 벡터화 진행
tfidf_matrix = vectorizer.fit_transform([processed_text])

# 단어 목록 정렬
feature_names = vectorizer.get_feature_names_out()
tfidf_values = tfidf_matrix.toarray().flatten()

# TF-IDF 값이 높은 상위 키워드 10개 추출
top_n = 10
sorted_indices = tfidf_values.argsort()[::-1][:top_n]
top_keywords = [(feature_names[i], tfidf_values[i]) for i in sorted_indices]

# 결과 출력 (표 형태)
df = pd.DataFrame(top_keywords, columns=['키워드', 'TF-IDF 값'])
print("\n상위 키워드:")
print(df)


상위 키워드:
      키워드  TF-IDF 값
0      파크  0.185695
1      대야  0.185695
2     12월  0.185695
3     42대  0.185695
4  7000가구  0.185695
5    774대  0.185695
6      가구  0.185695
7      경기  0.185695
8     경쟁률  0.185695
9      공급  0.185695
